<a href="https://colab.research.google.com/github/martinalegre77/python/blob/main/IA_tp2_LuisMartinAlegre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universidad Siglo 21
## Materia: Inteligencia Artificial
### Trabajo Práctico Nro 2
#### Alumno: Luis Martín Alegre

## 1. Implementar un proceso de búsqueda exhaustiva.

In [ ]:
def dfs(grafo, inicio, meta):
  """
  La función recibe tres argumentos:
  - el grafo
  - el nodo inicio (raíz)
  - el nodo objetivo (meta)
  """
  pila = [(inicio, [inicio])] # pila de nodos a recorrer
  visitados = set()  # nodos visitados

  while pila: # mientras la pila no esté vacía
    nodo, camino = pila.pop()

    if nodo == meta:
      # Si el nodo es igual a la meta devuelve el camino
      return camino

    if nodo not in visitados:
      visitados.add(nodo) # si el nodo no está en 'visitados' lo agrega
      vecinos = grafo[nodo] # expande el nodo

      for vecino in vecinos:
        pila.append((vecino, camino + [vecino])) # agrega los vecinos a la pila para recorrerlos

      # Si no encuentra la posición 'A' devuelve None
  return None

## Ejemplo con la posición de montaje 'A' del lado izquierdo (-B1, ..., -B6):

In [ ]:
grafo = {
     'B': ['B-1', 'B+1'],
     'B-1': ['B', 'B-2'],
     'B+1': ['B', 'B+2'],
     'B-2': ['B-3', 'B-1'],
     'B+2': ['B+3', 'B+1'],
     'B-3': ['B-2', 'B-4'],
     'B+3': ['B+2', 'B+4'],
     'B-4': ['B-5', 'B-3'],
     'B+4': ['B+5', 'B+3'],
     'B-5': ['B-6', 'B-4'],
     'B+5': ['B+6', 'B+4'],
     'B-6': ['B-5', 'A'], # 'A' después del nodo 'B-6' (lado izquierdo)
     'B+6': ['B+5'],
     'A': ['B-6'] # 'A' apunta a 'B-6'
 }

nodo_inicio = 'B'
nodo_meta = 'A'

resultado = dfs(grafo, nodo_inicio, nodo_meta)

if resultado:
    print(f"Se encontró un camino de {nodo_inicio} a {nodo_meta}: {resultado}")
else:
    print(f"No se encontró un camino de {nodo_inicio} a {nodo_meta}.")

Se encontró un camino de B a A: ['B', 'B-1', 'B-2', 'B-3', 'B-4', 'B-5', 'B-6', 'A']


## Ejemplo con la posición de montaje 'A' del lado derecho (+B1, ..., +B6):

In [ ]:
grafo = {
     'B': ['B-1', 'B+1'],
     'B-1': ['B', 'B-2'],
     'B+1': ['B', 'B+2'],
     'B-2': ['B-3', 'B-1'],
     'B+2': ['B+3', 'B+1'],
     'B-3': ['B-2', 'B-4'],
     'B+3': ['B+2', 'B+4'],
     'B-4': ['B-5', 'B-3'],
     'B+4': ['B+5', 'B+3'],
     'B-5': ['B-6', 'B-4'],
     'B+5': ['B+6', 'B+4'],
     'B-6': ['B-5'],
     'B+6': ['B+5', 'A'], # 'A' después del nodo 'B+6' (lado derecho)
     'A': ['B+6'] # 'A' apunta a 'B+6'
 }

nodo_inicio = 'B'
nodo_meta = 'A'

resultado = dfs(grafo, nodo_inicio, nodo_meta)

if resultado:
    print(f"Se encontró un camino de {nodo_inicio} a {nodo_meta}: {resultado}")
else:
    print(f"No se encontró un camino de {nodo_inicio} a {nodo_meta}.")

Se encontró un camino de B a A: ['B', 'B+1', 'B+2', 'B+3', 'B+4', 'B+5', 'B+6', 'A']


## 2. Implementar un proceso de búsqueda heurística.

In [44]:
class Nodo:
  """ Clase define los nodos(estados) del problema"""
  def __init__(self, nombre, h = 0):
    """Método que inicializa un nodo con sus atributos
		nombre = identificador del nodo(estado)
		heurística (h) = valor definido de la heurística del nodo
		vecinos = lista de los nodos con los que está conectado
		visitado = flag para saber si fue visitado o no
		padre = nodo padre
		costo = valor que tiene recorrerlo
		costoF = de f(n) = g(n) + h(n)"""
    self.nombre = nombre
    self.heuristica = h
    self.vecinos = []
    self.visitado = False
    self.padre = None
    self.costo = float('inf') # costo inicializado en infinito
    self.costoF = float('inf') # valor de f(n) inicializado en infinito

  def agregarVecino(self, nodo, costo = 0):
    """Método que agrega los nodos que se encuentren conectados a la lista
    de vecinos de un nodo. Antes revisa si aún no se encuentra en la lista
    de vecinos"""
    if nodo not in self.vecinos:
      self.vecinos.append([nodo, costo])

class Grafica:
  """Clase que define los nodos (estados) de la gráfica"""
  def __init__(self):
    """nodos = diccionario con los nodos de la grafica"""
    self.nodos = {}

  def agregarNodo(self, nombre, h = 0):
    """Método que agrega los nodos, recibiendo el nombre y la heuristica,
    revisando antes si no existe en el diccionario de nodos"""
    if nombre not in self.nodos:
      self.nodos[nombre] = Nodo(nombre, h)

  def unirNodo(self, a, b, costo = 0):
    """Método que une a los nodos, recibiendo el nombre de los mismos (a, b) y
    revisando si existen en la lista de nodos, además recibe el 'costo' del
    cambio de nodo, que se asigna a ambos por medio del método agregar vecino"""
    if a in self.nodos and b in self.nodos:
      self.nodos[a].agregarVecino(b, costo)
      self.nodos[b].agregarVecino(a, costo)

  def imprimirGrafica(self):
    """Método que imprime el gráfo completo con todos los valores
    (costo y heurística)"""
    for nodo in self.nodos:
      print(f"El costo del nodo {str(self.nodos[nodo].nombre)} con heurística {str(self.nodos[nodo].heuristica)} es {str(self.nodos[nodo].costo)} llegando desde {str(self.nodos[nodo].padre)}.")

  def camino(self, a, b):
    """Método que va guardando en la lista 'camino' los nodos en el orden que
    son visitados y actualizandola con los nodos con el menor costo"""
    camino = []
    actual = b
    while actual != None:
      camino.insert(0, actual)
      actual = self.nodos[actual].padre
    return camino

  def minimoH(self, lista):
    """Método que recibe la lista de los nodos no visitados, revisa si su
    longitud es mayor a cero (indica que aún hay nodos sin visitar), y
    realiza comparaciones de los costos de cada nodo para encontrar el de
    menor costo"""
    if len(lista) > 0:
      m = self.nodos[lista[0]].costoF
      nodo = lista[0]
      for e in lista:
        if m > self.nodos[e].costoF:
          m = self.nodos[e].costoF
          nodo = e
      return nodo
    return nodo

  def aEstrella(self, a, b):
    """Método con el algoritmo de búsqueda A*"""
    if a in self.nodos and b in self.nodos:
      self.nodos[a].costo = 0
      self.nodos[a].costoF = self.nodos[a].heuristica

      for nodo in self.nodos:
        if nodo != a:
          self.nodos[nodo].costo = float('inf')
          self.nodos[nodo].costoF = float('inf')
        self.nodos[nodo].padre = None

      abierto = [a]

      while len(abierto) > 0:
        actual = self.minimoH(abierto)

        if actual == b:
          return [self.camino(a, b), self.nodos[b].costo]

        abierto.remove(actual)
        self.nodos[actual].visitado = True

        for v in self.nodos[actual].vecinos:
          if self.nodos[v[0]].visitado == False:
            if self.nodos[v[0]].nombre not in abierto:
              abierto.append(v[0])
            if self.nodos[actual].costo + v[1] < self.nodos[v[0]].costo:
              self.nodos[v[0]].padre = actual
              self.nodos[v[0]].costo = self.nodos[actual].costo + v[1]
              self.nodosostoF = self.nodos[v[0]].costo + self.nodos[v[0]].heuristica
      return False
    else:
      return False

In [47]:
class main:
  """Clase para crear el grafo"""
	# Grafo
  block = Grafica()
  block.agregarNodo('B', 7)
  block.agregarNodo('B-1', 6)
  block.agregarNodo('B-2', 5)
  block.agregarNodo('B-3', 4)
  block.agregarNodo('B-4', 3)
  block.agregarNodo('B-5', 2)
  block.agregarNodo('B-6', 1)
  block.agregarNodo('A', 0)
  block.agregarNodo('B+1', 8)
  block.agregarNodo('B+2', 9)
  block.agregarNodo('B+3', 10)
  block.agregarNodo('B+4', 11)
  block.agregarNodo('B+5', 12)
  block.agregarNodo('B+6', 13)
  block.unirNodo('B', 'B-1', 1)
  block.unirNodo('B', 'B+1', 1)
  block.unirNodo('B-1', 'B-2', 1)
  block.unirNodo('B-2', 'B-3', 1)
  block.unirNodo('B-3', 'B-4', 1)
  block.unirNodo('B-4', 'B-5', 1)
  block.unirNodo('B-5', 'B-6', 1)
  block.unirNodo('B-6', 'A', 1)
  block.unirNodo('B+1', 'B+2', 1)
  block.unirNodo('B+2', 'B+3', 1)
  block.unirNodo('B+3', 'B+4', 1)
  block.unirNodo('B+4', 'B+5', 1)
  block.unirNodo('B+5', 'B+6', 1)
  print("\nLa movimiento de menor costo para ir de la posición 'B' a la 'A' es:")
  print(block.aEstrella('B', 'A'))
  print("\nLos valores de la gráfica son los siguientes:")
  block.imprimirGrafica()


La movimiento de menor costo para ir de la posición 'B' a la 'A' es:
[['B', 'B-1', 'B-2', 'B-3', 'B-4', 'B-5', 'B-6', 'A'], 7]

Los valores de la gráfica son los siguientes:
El costo del nodo B con heurística 7 es 0 llegando desde None.
El costo del nodo B-1 con heurística 6 es 1 llegando desde B.
El costo del nodo B-2 con heurística 5 es 2 llegando desde B-1.
El costo del nodo B-3 con heurística 4 es 3 llegando desde B-2.
El costo del nodo B-4 con heurística 3 es 4 llegando desde B-3.
El costo del nodo B-5 con heurística 2 es 5 llegando desde B-4.
El costo del nodo B-6 con heurística 1 es 6 llegando desde B-5.
El costo del nodo A con heurística 0 es 7 llegando desde B-6.
El costo del nodo B+1 con heurística 8 es 1 llegando desde B.
El costo del nodo B+2 con heurística 9 es 2 llegando desde B+1.
El costo del nodo B+3 con heurística 10 es 3 llegando desde B+2.
El costo del nodo B+4 con heurística 11 es 4 llegando desde B+3.
El costo del nodo B+5 con heurística 12 es 5 llegando desde B+

# El camino y los valores obtenidos son estimados con la posición 'A' del lado izquierdo.